<a href="https://colab.research.google.com/github/AndrewAOh/HitModel/blob/main/Updated_HitModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Data

In [51]:
import pandas as pd
import glob
import os
import numpy as np

df2 = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Combined Data/23_NCAA_Dataset*.csv")

<ipython-input-51-5db35e9a50b0>:6: DtypeWarning: Columns (27,92,93,94,167) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/content/drive/Shareddrives/USC Baseball/Combined Data/23_NCAA_Dataset*.csv")


# Clean Up Dataset & Recode Variables

In [52]:
cleanD1 = df2[df2['Level'] == "D1"]

In [53]:
BIPdf = cleanD1[cleanD1['PitchCall'].str.contains("InPlay") == True]
BIPdf = BIPdf[BIPdf['TaggedPitchType'].str.contains("Undefined") == False]

df = BIPdf

In [54]:
from sklearn.model_selection import train_test_split
import matplotlib as plt
from sklearn import datasets
from sklearn import metrics


# creates list of conditions that determine whether PlayResult is a hit or not
conditions = [
    (df['PlayResult'] == 'Single'),
    (df['PlayResult'] == 'Double'),
    (df['PlayResult'] == 'Triple'),
    (df['PlayResult'] == 'HomeRun'),
    (df['PlayResult'] == 'Out'),
    (df['PlayResult'] == 'Error'),
    (df['PlayResult'] == 'Sacrifice'),
    (df['PlayResult'] == 'FieldersChoice')
]
# creates a list of values to assign for hit (1) or not (0)
values = [1, 1, 1, 1, 0, 0, 0, 0]
# creates new column determining hit or not and assign values to it based on the conditions
df['isHit'] = np.select(conditions, values)

<ipython-input-54-9e151446c46b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isHit'] = np.select(conditions, values)


In [55]:
# Recodes pitch types into 5 different buckets

# sinkers, 4 seam, 2 seam, 1 seam = fastball
# cutters = sliders
# splitter = changeup

# creates list of conditions that determine whether TaggedPitchType is fastball/slider/changeup/curveball
conditions = [
    (df['TaggedPitchType'].str.contains('Fastball|FourSeamFastBall|TwoSeamFastBall|OneSeamFastBall|Sinker')),  # fastballs
    (df['TaggedPitchType'].str.contains('Slider|Cutter')),  # sliders
    (df['TaggedPitchType'].str.contains('ChangeUp|Splitter')),  # changeups
    (df['TaggedPitchType'].str.contains('Curveball')) # curveballs
]

# creates a list of values to assign for pitch types
values = [0, 1, 2, 3]
# creates new column determining pitch type and assign values to it based on the conditions; assigns 4 if an other pitch
df['pitchType'] = np.select(conditions, values, default = 4)

In [56]:
# Recodes Pitch Handedness to binary variable

conditions = [
    (df['PitcherThrows'] == 'Right'),  # RHP
    (df['PitcherThrows'] == 'Left')  # LHP
]

# creates a list of values to assign for pitch handedness where RHP = 1 & LHP = 0
values = [0, 1]
# creates new column determining pitch hand
df['pitcherHand'] = np.select(conditions, values)

In [57]:
# Recodes Hitting Handedness to binary variable

conditions = [
    (df['BatterSide'] == 'Right'),  # RHB
    (df['BatterSide'] == 'Left')  # LHB
]

# creates a list of values to assign for hitter handedness where RHB = 1 & LHB = 0
values = [0, 1]
# creates new column determining hit hand
df['hitterHand'] = np.select(conditions, values)

In [58]:
# Calculates x & y position of the ball hit
df['xSpray'] = np.sin(np.radians(df['Bearing'])) * df['Distance']
df['ySpray'] = np.cos(np.radians(df['Bearing'])) * df['Distance']

# Following Code Identifies Important Variables

Finds correlations between variables to reduce multicollinearity

In [27]:
df4 = df[['pitchType', 'pitcherHand', 'hitterHand', 'ExitSpeed', 'Angle', 'xSpray', 'ySpray', 'HitSpinRate', 'Distance', 'Bearing', 'HangTime', 'MaxHeight', 'ContactPositionX', 'ContactPositionY', 'ContactPositionZ', 'isHit']].dropna()
df4.corr()

,pitchType,pitcherHand,hitterHand,ExitSpeed,Angle,xSpray,ySpray,HitSpinRate,Distance,Bearing,HangTime,MaxHeight,ContactPositionX,ContactPositionY,ContactPositionZ,isHit
pitchType,1.000000,0.014784,0.006894,-0.054404,-0.021890,-0.020308,-0.046623,-0.021545,-0.043874,-0.023360,-0.040685,-0.039255,0.238123,-0.157831,0.035174,-0.001338
pitcherHand,0.014784,1.000000,-0.066501,-0.021538,0.000436,-0.002298,-0.013664,-0.002640,-0.013495,-0.002937,-0.007999,-0.005740,0.006344,-0.010440,0.139447,-0.001459
hitterHand,0.006894,-0.066501,1.000000,0.013272,0.003759,0.057625,0.005250,-0.005699,0.006744,0.102610,0.001993,0.004643,-0.019458,-0.041064,-0.242510,0.001092
ExitSpeed,-0.054404,-0.021538,0.013272,1.000000,-0.102490,-0.029571,0.373549,-0.129139,0.377794,-0.025962,0.132667,0.120895,-0.036870,-0.052064,0.016598,0.222650
Angle,-0.021890,0.000436,0.003759,-0.102490,1.000000,0.019255,0.623747,0.453143,0.650527,0.052909,0.913747,0.913534,0.080786,0.110225,0.008145,-0.276471
xSpray,-0.020308,-0.002298,0.057625,-0.029571,0.019255,1.000000,-0.008722,0.095147,-0.012475,0.898454,0.017476,0.014814,-0.096928,-0.017216,0.238569,-0.047676
ySpray,-0.046623,-0.013664,0.005250,0.373549,0.623747,-0.008722,1.000000,0.092480,0.981841,0.029574,0.847354,0.695575,0.039882,0.036888,0.024580,-0.037981
HitSpinRate,-0.021545,-0.002640,-0.005699,-0.129139,0.453143,0.095147,0.092480,1.000000,0.129453,0.099566,0.376682,0.442866,-0.034778,0.029764,0.017156,-0.208308
Distance,-0.043874,-0.013495,0.006744,0.377794,0.650527,-0.012475,0.981841,0.129453,1.000000,0.026572,0.863695,0.709563,0.046072,0.040709,0.021436,-0.010108
Bearing,-0.023360,-0.002937,0.102610,-0.025962,0.052909,0.898454,0.029574,0.099566,0.026572,1.000000,0.054072,0.044950,-0.103168,-0.021033,0.243013,-0.051301


In [28]:
df6 = df4.corr()
csv = df6.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/HitCorr*.csv", index=False)

Determines the Mean, STD, Median, 25th %, and 75th% of Variables Grouped by Hit or Not Hit

In [29]:
import statistics

# 'target_column' is the name of the column with 0s and 1s, and 'predictor_columns' is a list of predictor column names.

def compute_statistics_by_target(df, target_column, predictor_columns):
    # Group the dataframe by the target variable and calculate statistics for each group
    grouped = df.groupby(target_column)

    # Initialize a dictionary to store the statistics for each group
    statistics_dict = {}

    for target_value, group in grouped:
        # Calculate the means for each predictor column
        means = group[predictor_columns].mean()

        # Calculate the standard deviations for each predictor column
        std_devs = group[predictor_columns].std()

        # Calculate the medians for each predictor column
        medians = group[predictor_columns].median()

        # Calculate the 25th percentile for each predictor column
        percentile_25 = group[predictor_columns].quantile(0.25)

        # Calculate the 75th percentile for each predictor column
        percentile_75 = group[predictor_columns].quantile(0.75)

        # Combine the means and standard deviations for each group into a single row
        row = means.tolist() + std_devs.tolist() + medians.tolist() + percentile_25.tolist() + percentile_75.tolist()

        # Add the row to the statistics dictionary
        statistics_dict[target_value] = row

    # Convert the dictionary to a dataframe and set the index
    statistics_df = pd.DataFrame.from_dict(statistics_dict, orient='index', columns=predictor_columns +
                                        [f'Std_Dev_{col}' for col in predictor_columns] +
                                        [f'Median_{col}' for col in predictor_columns] +
                                        [f'Percentile_25_{col}' for col in predictor_columns] +
                                        [f'Percentile_75_{col}' for col in predictor_columns])


    return statistics_df

# Example usage:
target_column_name = 'isHit'
predictor_columns_list = ['pitchType', 'pitcherHand', 'hitterHand', 'ExitSpeed', 'Angle', 'xSpray', 'ySpray', 'HitSpinRate', 'Distance', 'Bearing', 'HangTime', 'MaxHeight', 'ContactPositionX', 'ContactPositionY', 'ContactPositionZ']
statistics_table = compute_statistics_by_target(df, target_column_name, predictor_columns_list)

print(statistics_table)

   pitchType  pitcherHand  hitterHand  ExitSpeed      Angle    xSpray  \
0   0.629922     0.249810    0.358557  82.106823  11.552601  1.182637   
1   0.606346     0.251782    0.366435  91.715885  10.322717 -6.877970   

       ySpray  HitSpinRate    Distance   Bearing  ...  Percentile_75_xSpray  \
0  136.510891  3156.021696  147.286147 -0.262379  ...             19.041240   
1  178.030777  2569.678787  196.797005 -2.651991  ...             48.185581   

   Percentile_75_ySpray  Percentile_75_HitSpinRate  Percentile_75_Distance  \
0            264.872161                4127.318907               292.04475   
1            264.386196                3240.623878               292.75192   

   Percentile_75_Bearing  Percentile_75_HangTime  Percentile_75_MaxHeight  \
0              17.257073                4.634043                86.959160   
1              17.876045                3.439022                40.862295   

   Percentile_75_ContactPositionX  Percentile_75_ContactPositionY  \
0     

In [30]:
csv = statistics_table.to_csv("/content/drive/Shareddrives/USC Baseball/ML Stuff/HitANOVA*.csv", index=False)

# Making ML Model

In [61]:
df['isHit'].value_counts()

0    209275
1    115322
Name: isHit, dtype: int64

In [64]:
minority_class = df[df['isHit'] == 1]
majority_class = df[df['isHit'] == 0]

print("majority class is: " + str(len(majority_class)))
print("minority class is: " + str(len(minority_class)))

print("total is: " + str(len(majority_class)+len(minority_class)))

majority class is: 209275
minority class is: 115322
total is: 324597


In [170]:
cleandf = df.dropna(subset=['ExitSpeed', 'Angle', 'xSpray', 'ySpray',  'MaxHeight', 'pitchType', 'pitcherHand', 'hitterHand', 'isHit'])

In [171]:
cleandf['isHit'].value_counts()

0    174684
1    108494
Name: isHit, dtype: int64

In [172]:
minority_class = cleandf[cleandf['isHit'] == 1]
majority_class = cleandf[cleandf['isHit'] == 0]

print("majority class is: " + str(len(majority_class)))
print("minority class is: " + str(len(minority_class)))

print("total is: " + str(len(majority_class)+len(minority_class)))

# Retains 87.23987% of data

majority class is: 174684
minority class is: 108494
total is: 283178


In [173]:
# resamples the class to have a even distribution of hits and non-hits
# implements downsample on majority class (non-hits) and oversample on minority class (hits)
# Idea: bring to the average of minority class & majority class

average = int((len(majority_class) + len(minority_class))/2)

from sklearn.utils import resample
majority_class_downsampled = resample(majority_class,
                                      replace=False,  # sample without replacement
                                      n_samples=average,  # match average size
                                      random_state=42)
minority_class_oversampled = resample(minority_class,
                                      replace=True,
                                      n_samples=average,
                                      random_state = 42)


# Combine minority class and downsampled majority class
balanced_df = pd.concat([minority_class_oversampled, majority_class_downsampled])

In [174]:
balanced_df = balanced_df.sample(frac=1, random_state=42)

In [175]:
rel_data = balanced_df[['ExitSpeed', 'Angle', 'xSpray', 'ySpray', 'MaxHeight', 'pitchType', 'pitcherHand', 'hitterHand']].to_numpy()
classification = balanced_df['isHit'].to_numpy()

In [176]:
minority = balanced_df[balanced_df['isHit'] == 1]
majority = balanced_df[balanced_df['isHit'] == 0]

print("majority class is: " + str(len(majority)))
print("minority class is: " + str(len(minority)))

print("total is: " + str(len(majority)+len(minority)))

majority class is: 141589
minority class is: 141589
total is: 283178


In [177]:
trainNum = int((len(majority) + len(minority))*0.7)

y = np.unique(classification, return_inverse=True)
y_train = y[1][:trainNum]
y_test = y[1][trainNum:]

x = np.unique(classification, return_inverse=True)
x_train = rel_data[:trainNum]
x_test = rel_data[trainNum:]

In [178]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(x_train)
X_scaled = scaler.transform(x_train)

In [179]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

names = [
    "Decision Tree",
    "Random Forest"
]

classifiers = [
    #DecisionTreeClassifier(max_depth=50),
    #DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=None, n_estimators=50)
]

In [180]:
accuracies = []
for opt in classifiers:
  clf = make_pipeline(StandardScaler(), opt)
  clf.fit(x_train, y_train)

  y_pred = clf.predict((x_test))
  accuracies.append(metrics.accuracy_score(y_test, y_pred))

In [181]:
from sklearn.metrics import classification_report
target_names = ['not hit', 'hit']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     not hit       0.95      0.94      0.94     42311
         hit       0.94      0.95      0.95     42643

    accuracy                           0.95     84954
   macro avg       0.95      0.95      0.95     84954
weighted avg       0.95      0.95      0.95     84954



In [161]:
print(accuracies)
# Old model; loses too much data because of 'HitSpinRate' column
# Variables:'ExitSpeed', 'Angle', 'xSpray', 'ySpray', 'HitSpinRate', 'MaxHeight', 'pitchType', 'pitcherHand', 'hitterHand'

[0.9492242844998583]


In [182]:
print("New variable features:")
# Final Model Accuracy: 94.502907%
print(accuracies)

New variable features:
[0.9450290745579961]


# Export Final Model

In [183]:
import joblib
from joblib import dump

import os
os.chdir('/content/drive/Shareddrives/USC Baseball/SabermetricFiles')

# export the trained model to disk
joblib.dump(clf, 'batting_avg_model4.joblib')

['batting_avg_model4.joblib']